In [1]:
!pip install -r ./requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('churchill_speech.txt') as f:
  churchill_speech = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20, length_function=len)
chunks = text_splitter.create_documents([churchill_speech])

In [10]:
print(chunks[0])

page_content='Winston Churchill Speech - We Shall Fight on the Beaches
We Shall Fight on the Beaches
June 4, 1940
House of Commons
From the moment that the French defenses at Sedan and on the Meuse were broken at the end of the
second week of May, only a rapid retreat to Amiens and the south could have saved the British and
French Armies who had entered Belgium at the appeal of the Belgian King; but this strategic fact was
not immediately realized. The French High Command hoped they would be able to close the gap, and
the Armies of the north were under their orders. Moreover, a retirement of this kind would have
involved almost certainly the destruction of the fine Belgian Army of over 20 divisions and the
abandonment of the whole of Belgium. Therefore, when the force and scope of the German
penetration were realized and when a new French Generalissimo, General Weygand, assumed
command in place of General Gamelin, an effort was made by the French and British Armies in'


In [11]:
print(chunks[1])

page_content='Belgium to keep on holding the right hand of the Belgians and to give their own right hand to a newly
created French Army which was to have advanced across the Somme in great strength to grasp it.
However, the German eruption swept like a sharp scythe around the right and rear of the Armies of
the north. Eight or nine armored divisions, each of about four hundred armored vehicles of different
kinds, but carefully assorted to be complementary and divisible into small self-contained units, cut off
all communications between us and the main French Armies. It severed our own communications for
food and ammunition, which ran first to Amiens and afterwards through Abbeville, and it shore its
way up the coast to Boulogne and Calais, and almost to Dunkirk. Behind this armored and
mechanized onslaught came a number of German divisions in lorries, and behind them again there
plodded comparatively slowly the dull brute mass of the ordinary German Army and German people,'


Embedding Cost

In [14]:
def print_embedding_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model("text-embedding-ada-002")
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f"Total Tokens: {total_tokens}")
  print(f"Embedding Cost: ${total_tokens / 1000 * 0.0004:.6f}")

print_embedding_cost(chunks)

Total Tokens: 4665
Embedding Cost: $0.001866


In [15]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [18]:
vector = embedding.embed_query(chunks[0].page_content)
# print(vector)

Inserting Embeddings into Pincone Index

In [19]:
import pinecone
from langchain.vectorstores import Pinecone

pc = pinecone.Pinecone()

In [20]:
for i in pc.list_indexes().names():
  print("Deleting all indexes")
  pc.delete_index(i)

Deleting all indexes


In [21]:
index_name = 'churchill-speech'
if index_name not in pc.list_indexes().names():
  print(f"Creating index {index_name}...")
  pc.create_index(name=index_name, dimension=1536, metric='cosine', spec=pinecone.PodSpec(environment='gcp-starter'))
  print("Done")

Creating index churchill-speech...
Done
